In [17]:
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import glob
import os
import string

start_date = np.datetime64('2024-09-25 00:00:00')
end_date = np.datetime64('2024-09-28 00:00:00')

# read wave results
# Maitane model results
#url='http://geoport.whoi.edu/thredds/dodsC/vortexfs1/usgs/Projects/dorian/dorian_coupled11/PAM/dorian_his.ncml'
url ='http://icoast.rc.ufl.edu/thredds/dodsC/Noppcoawsthindcast/Helene/ocean_his.nc'
dsm = xr.open_dataset(url)
latm = dsm.lat_rho.values
lonm = dsm.lon_rho.values
time_all = dsm.ocean_time.values
idxms = int( np.where(time_all >= start_date)[0][0] )
idxme = int( np.where(time_all <= end_date)[0][-1] )
timem = time_all[idxms:idxme]
mask_rho = dsm.mask_rho.values
# TODO - change the time of the wet-dry mast
#wd_mask_rho = dsm.wetdry_mask_rho[19,:,:].values
wd_mask_rho = dsm.mask_rho[:,:].values

# Find closest model grid point
# Calculate the difference between the model grid points and the buoy locations; find model indices to closest points
model_shape = np.shape( lonm )
print('model shape: ',model_shape)
print('nlat, nlon: ',np.shape(mask_rho))


model shape:  (545, 690)
nlat, nlon:  (545, 690)


In [18]:
dsm

<xarray.Dataset> Size: 29GB
Dimensions:          (tracer: 2, boundary: 4, s_rho: 30, s_w: 31, eta_rho: 545,
                      xi_rho: 690, eta_u: 545, xi_u: 689, eta_v: 544,
                      xi_v: 690, eta_psi: 544, xi_psi: 689, ocean_time: 41)
Coordinates:
  * s_rho            (s_rho) float64 240B -0.9833 -0.95 ... -0.05 -0.01667
  * s_w              (s_w) float64 248B -1.0 -0.9667 -0.9333 ... -0.03333 0.0
    lon_rho          (eta_rho, xi_rho) float64 3MB -87.07 -87.05 ... -77.74
    lat_rho          (eta_rho, xi_rho) float64 3MB 20.83 20.84 ... 34.55 34.56
    lon_u            (eta_u, xi_u) float64 3MB ...
    lat_u            (eta_u, xi_u) float64 3MB ...
    lon_v            (eta_v, xi_v) float64 3MB ...
    lat_v            (eta_v, xi_v) float64 3MB ...
    lon_psi          (eta_psi, xi_psi) float64 3MB ...
    lat_psi          (eta_psi, xi_psi) float64 3MB ...
  * ocean_time       (ocean_time) datetime64[ns] 328B 2024-09-24 ... 2024-09-29
Dimensions without coordinates: tracer, boundary, eta_rho, xi_rho, eta_u, xi_u,
                                eta_v, xi_v, eta_psi, xi_psi
Data variables: (12/127)
    ntimes           int32 4B ...
    ndtfast          int32 4B ...
    dt               float64 8B ...
    dtfast           float64 8B ...
    dstart           datetime64[ns] 8B ...
    nHIS             int32 4B ...
    ...               ...
    Lwavep           (ocean_time, eta_rho, xi_rho) float32 62MB ...
    Dwave            (ocean_time, eta_rho, xi_rho) float32 62MB ...
    Pwave_top        (ocean_time, eta_rho, xi_rho) float32 62MB ...
    Dissip_fric      (ocean_time, eta_rho, xi_rho) float32 62MB ...
    Dissip_break     (ocean_time, eta_rho, xi_rho) float32 62MB ...
    Dissip_wcap      (ocean_time, eta_rho, xi_rho) float32 62MB ...
Attributes: (12/34)
    file:                            ocean_his.nc
    format:                          netCDF-3 64bit offset file
    Conventions:                     CF-1.4, SGRID-0.3
    type:                            ROMS/TOMS history file
    title:                           NHCI-COAWST FORECAST
    var_info:                        ./../../coawst_code/COAWST_05292024/ROMS...
    ...                              ...
    tiling:                          016x008
    history:                         ROMS/TOMS, Version 4.1, Sunday - Novembe...
    ana_file:                        ROMS/Functionals/ana_btflux.h, ROMS/Func...
    CPP_options:                     FORECAST, ADD_FSOBC, ANA_BSFLUX, ANA_BTF...
    DODS_EXTRA.Unlimited_Dimension:  ocean_time
    EXTRA_DIMENSION.N:               30

In [19]:
h = dsm['h'].values
lat = dsm['lat_rho'].values
lon = dsm['lon_rho'].values

In [ ]:
plt.scatter